In [47]:
#imports

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from urllib.request import urlopen
from bs4 import BeautifulSoup
from sys import platform
import io
import datetime
import requests
import sys
import ephem #might need to run pip3 install ephem

In [76]:
if platform == "linux" or platform == "linux2":
    data_string = "/home/maria/Documents/data/pied_piper/skagit/"
    output_string = "/home/maria/Documents/output/pied_piper/skagit/"

elif platform == "win32"or platform == "win64":
    data_string = "C:/Users/maria/OneDrive/Documents/data/pied_piper/"
    output_string = "C:/Users/maria/OneDrive/Documents/output/pied_piper/skagit/"
# print(sys.path)

In [15]:

try:
    df_2010 = pd.read_excel(
   io= data_string + "2010 Skagit mainstem catch.xls",
   sheet_name ='Screw',
    header = [1,2]
)
except PermissionError:
    print("Permission Error. Close the file, if it is open.")
    

df_2010.head()
columns = list(df_2010)
print(columns)

df_2010.columns  = ['_'.join(col) for col in df_2010.columns.values]
df_2010 = df_2010.rename(columns=lambda x: x.strip()) #remove extra trailing spaces
print(df_2010.columns)

df_2010 = df_2010.rename(columns={'Unnamed: 0_level_0_Page #': "Page",
                                  'Unnamed: 1_level_0_Date Dn': "Date",
                                  'Unnamed: 2_level_0_Time Dn': "Down_time",
                                  'Unnamed: 4_level_0_Time Up': "Up_time",
                                 'Unnamed: 3_level_0_Date Up': "Up_date",
                                 'Unnamed: 5_level_0_Stat Week':'Week'})

# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#     display(df_2010)
    

[('Unnamed: 0_level_0', 'Page #'), ('Unnamed: 1_level_0', 'Date Dn'), ('Unnamed: 2_level_0', 'Time Dn'), ('Unnamed: 3_level_0', 'Date Up'), ('Unnamed: 4_level_0', 'Time Up'), ('Unnamed: 5_level_0', 'Stat Week'), ('   Wild Chin 0+', 'Total'), ('   Wild Chin 0+', 'Mort'), ('Hatch Chin 0+', 'Total'), ('Hatch Chin 0+', 'Sac'), ('Hatch Chin 0+', 'Mort'), ('Hatch Chin 0+', 'Unmk'), ('   Wild Chin 1+', 'Total'), ('   Wild Chin 1+', 'Mort'), ('Hatch Chin 1+', 'Total'), ('Hatch Chin 1+', 'Mort'), ('Hatch Chin 1+', 'Ad + CWT'), ('Hatch Chin 1+', 'Ad'), ('Hatch Chin 1+', 'unmk CWT'), ('Hatch Chin 1+', 'unmk/ NT'), ('        Chum', 'Total'), ('        Chum', 'Mort'), ('Coho 0+', 'Total'), ('Coho 0+', 'Sac'), ('Coho 0+', 'Mort'), ('               Wild Coho 1+', 'Total'), ('               Wild Coho 1+', 'LV'), ('               Wild Coho 1+', 'Baker'), ('               Wild Coho 1+', 'Sac'), ('               Wild Coho 1+', 'Mort'), ('  Hatch Coho 1+', 'Total'), ('  Hatch Coho 1+', 'Mort'), ('        

In [16]:
# format the date and time rows

df_2010['Up_time'] = pd.to_datetime(df_2010['Up_time'], format='%H%M').dt.time
df_2010['Down_time'] = pd.to_datetime(df_2010['Down_time'], format='%H%M').dt.time

#remove last 3 rows
df_2010 = df_2010.iloc[:-3]

# #make columns that are consistent with previous years
# df_2010['Up'] = 
df_2010['Up'] = pd.to_datetime((df_2010['Up_date'].astype(str)) + ' ' + (df_2010["Up_time"].astype(str)))
df_2010['Down'] = pd.to_datetime((df_2010['Date'].astype(str)) + ' ' + (df_2010["Down_time"].astype(str)))
df_2010['midpoint'] = df_2010['Down'] + (df_2010['Up'] - df_2010['Down'])/2
df_2010['In'] = df_2010['Up'] - df_2010['Down']

df_2010['In'] = df_2010['In']/ np.timedelta64(1, 'h')

# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#     display(df_2010)

In [17]:
df_2010.columns

df_year = df_2010.rename(columns={'Wild Chin 0+_Total': "chinook0_wild_num",
                                 'Hatch Chin 0+_Total': "chinook0_hatchery_num",
                                 'Wild Chin 1+_Total': "chinook1_wild_num",
                                 'Hatch Chin 1+_Total': "chinook1_hatchery_num",
                                  'Coho 0+_Total':'coho0_wild_total',
                                  'Wild Coho 1+_Total':'coho1_wild_num',
                                  'Hatch Coho 1+_Total': "coho1_hatchery_num",
                                  'Wild Sthd_Total':'steelheadsmolt_wild_num',
                                  'Pink_Total': 'pink0_wild_num',
                                 'Chum_Total': 'chum0_wild_num'})
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#     display(df_year)

In [18]:
df_year_subset = df_year[['Date', 'Down_time', 'Up_date', 'Up_time', 'midpoint', 'In', 'Up', 'Down',
                         'chinook0_hatchery_num', 'chinook0_wild_num',
                         'chinook1_hatchery_num', 'chinook1_wild_num',
                         'coho1_hatchery_num', 'coho1_wild_num',
                         'chum0_wild_num', 'pink0_wild_num','steelheadsmolt_wild_num']]

#convet nans to 0s
df_year_subset.fillna(0, inplace=True)


with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(df_year_subset)

C:\Users\maria\anaconda3\lib\site-packages\pandas\core\frame.py:5176: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


,Date,Down_time,Up_date,Up_time,midpoint,In,Up,Down,chinook0_hatchery_num,chinook0_wild_num,chinook1_hatchery_num,chinook1_wild_num,coho1_hatchery_num,coho1_wild_num,chum0_wild_num,pink0_wild_num,steelheadsmolt_wild_num
0,2010-01-21,16:45:00,2010-01-22,08:30:00,2010-01-22 00:37:30,15.750000,2010-01-22 08:30:00,2010-01-21 16:45:00,0.0,6.0,0.0,0.0,0.0,0.0,0.0,30.0,0.0
1,2010-01-22,08:40:00,2010-01-22,17:15:00,2010-01-22 12:57:30,8.583333,2010-01-22 17:15:00,2010-01-22 08:40:00,0.0,4.0,0.0,0.0,0.0,0.0,12.0,100.0,0.0
2,2010-01-22,17:25:00,2010-01-23,07:45:00,2010-01-23 00:35:00,14.333333,2010-01-23 07:45:00,2010-01-22 17:25:00,0.0,9.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
3,2010-01-23,17:30:00,2010-01-24,08:00:00,2010-01-24 00:45:00,14.500000,2010-01-24 08:00:00,2010-01-23 17:30:00,0.0,14.0,0.0,0.0,0.0,0.0,0.0,29.0,0.0
4,2010-01-24,17:00:00,2010-01-25,07:30:00,2010-01-25 00:15:00,14.500000,2010-01-25 07:30:00,2010-01-24 17:00:00,0.0,12.0,0.0,1.0,0.0,0.0,2.0,17.0,0.0
5,2010-01-25,07:40:00,2010-01-25,17:00:00,2010-01-25 12:20:00,9.333333,2010-01-25 17:00:00,2010-01-25 07:40:00,0.0,2.0,0.0,0.0,0.0,0.0,6.0,38.0,0.0
6,2010-01-25,17:10:00,2010-01-26,07:30:00,2010-01-26 00:20:00,14.333333,2010-01-26 07:30:00,2010-01-25 17:10:00,0.0,8.0,0.0,0.0,0.0,0.0,3.0,14.0,0.0
7,2010-01-26,17:00:00,2010-01-27,07:30:00,2010-01-27 00:15:00,14.500000,2010-01-27 07:30:00,2010-01-26 17:00:00,0.0,6.0,0.0,0.0,0.0,0.0,2.0,19.0,0.0
8,2010-01-27,17:30:00,2010-01-28,07:45:00,2010-01-28 00:37:30,14.250000,2010-01-28 07:45:00,2010-01-27 17:30:00,0.0,5.0,0.0,0.0,0.0,0.0,1.0,9.0,0.0
9,2010-01-28,08:00:00,2010-01-28,17:30:00,2010-01-28 12:45:00,9.500000,2010-01-28 17:30:00,2010-01-28 08:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0


In [19]:
#checking whether the midpoint of the time when trap was in during day or night
def categorize_daytime(dt):
    if dt.hour >= 6 and dt.hour < 18:
        return 'Day'
    else:
        return 'Night'
df_year_subset['daytime_category'] = df_year_subset['midpoint'].apply(categorize_daytime)

C:\Users\maria\AppData\Local\Temp/ipykernel_24580/3821061683.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_year_subset['daytime_category'] = df_year_subset['midpoint'].apply(categorize_daytime)


In [65]:
df_day = df_year_subset[df_year_subset['daytime_category']=="Day"]
df_night = df_year_subset[df_year_subset['daytime_category']=="Night"]

#need to have one row for every date
df_dates_day = pd.DataFrame({'date_range' : 
                         pd.date_range(start='2010-01-20 12:00:00', 
                                       end='2010-07-22 12:00:00')})
df_dates_night = pd.DataFrame({'date_range' : 
                         pd.date_range(start='2010-01-20 00:00:00', 
                                       end='2010-07-22 00:00:00')})

df_merged_day = pd.merge_asof(df_dates_day, 
              df_day, left_on = 'date_range', 
              right_on = 'midpoint', direction = 'nearest',
              tolerance = pd.Timedelta(hours = 12))

df_merged_night = pd.merge_asof(df_dates_night, 
              df_night, left_on = 'date_range', 
              right_on = 'midpoint', direction = 'nearest',
              tolerance = pd.Timedelta(hours = 12))

In [66]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(df_merged_night)

,date_range,Date,Down_time,Up_date,Up_time,midpoint,In,Up,Down,chinook0_hatchery_num,chinook0_wild_num,chinook1_hatchery_num,chinook1_wild_num,coho1_hatchery_num,coho1_wild_num,chum0_wild_num,pink0_wild_num,steelheadsmolt_wild_num,daytime_category
0,2010-01-20,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2010-01-21,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2010-01-22,2010-01-21,16:45:00,2010-01-22,08:30:00,2010-01-22 00:37:30,15.750000,2010-01-22 08:30:00,2010-01-21 16:45:00,0.0,6.0,0.0,0.0,0.0,0.0,0.0,30.0,0.0,Night
3,2010-01-23,2010-01-22,17:25:00,2010-01-23,07:45:00,2010-01-23 00:35:00,14.333333,2010-01-23 07:45:00,2010-01-22 17:25:00,0.0,9.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,Night
4,2010-01-24,2010-01-23,17:30:00,2010-01-24,08:00:00,2010-01-24 00:45:00,14.500000,2010-01-24 08:00:00,2010-01-23 17:30:00,0.0,14.0,0.0,0.0,0.0,0.0,0.0,29.0,0.0,Night
5,2010-01-25,2010-01-24,17:00:00,2010-01-25,07:30:00,2010-01-25 00:15:00,14.500000,2010-01-25 07:30:00,2010-01-24 17:00:00,0.0,12.0,0.0,1.0,0.0,0.0,2.0,17.0,0.0,Night
6,2010-01-26,2010-01-25,17:10:00,2010-01-26,07:30:00,2010-01-26 00:20:00,14.333333,2010-01-26 07:30:00,2010-01-25 17:10:00,0.0,8.0,0.0,0.0,0.0,0.0,3.0,14.0,0.0,Night
7,2010-01-27,2010-01-26,17:00:00,2010-01-27,07:30:00,2010-01-27 00:15:00,14.500000,2010-01-27 07:30:00,2010-01-26 17:00:00,0.0,6.0,0.0,0.0,0.0,0.0,2.0,19.0,0.0,Night
8,2010-01-28,2010-01-27,17:30:00,2010-01-28,07:45:00,2010-01-28 00:37:30,14.250000,2010-01-28 07:45:00,2010-01-27 17:30:00,0.0,5.0,0.0,0.0,0.0,0.0,1.0,9.0,0.0,Night
9,2010-01-29,2010-01-28,17:40:00,2010-01-29,08:00:00,2010-01-29 00:50:00,14.333333,2010-01-29 08:00:00,2010-01-28 17:40:00,0.0,5.0,0.0,2.0,0.0,0.0,0.0,12.0,0.0,Night


In [70]:
df_merged_day['midpoint_doy'] = df_merged_day['midpoint'].dt.dayofyear
df_merged_day['doy'] = df_merged_day['date_range'].dt.dayofyear
df_merged_day['year'] = df_merged_day['date_range'].dt.year
df_merged_day['daytime_category'] = 'day'
df_merged_day['Date'] = pd.to_datetime(df_merged_day['date_range'].dt.date)

df_merged_night['midpoint_doy'] = df_merged_night['midpoint'].dt.dayofyear
df_merged_night['doy'] = df_merged_night['date_range'].dt.dayofyear
df_merged_night['year'] = df_merged_night['date_range'].dt.year
df_merged_night['daytime_category'] = 'night'
df_merged_night['Date'] = pd.to_datetime(df_merged_night['date_range'].dt.date)

In [29]:
#getting temp and flow data from USGS

year = '2010'
url = 'https://waterdata.usgs.gov/nwis/dv?cb_00010=on&cb_00060=on&cb_00065=on&format=rdb&site_no=12178000&legacy=&referred_module=sw&period=&begin_date=2010-01-01&end_date=2010-12-16'

html = urlopen(url).read()
soup = BeautifulSoup(html, features="html.parser")

# kill all script and style elements
for script in soup(["script", "style"]):
    script.extract()    # rip it out

# get text
text = soup.get_text()
text_data = text[1945:]
df = pd.read_csv(io.StringIO(text_data), sep = '\t', header = None, names = ["USGS", "random0", "Date", "flow", "random1",
                                                                       "flow_height", "random2","temp_max", "random3", 
                                                                       "temp_min", "random4","temp", "random5"])

df_temp_flow_subset = df[['Date','flow','temp']]
df_temp_flow_subset['Date'] = pd.to_datetime(df_temp_flow_subset['Date'])
# df_temp_flow_subset = df_temp_flow_subset.rename(columns={'temp_mean':'temp',})
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(df_temp_flow_subset)


C:\Users\maria\AppData\Local\Temp/ipykernel_24580/2747207474.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp_flow_subset['Date'] = pd.to_datetime(df_temp_flow_subset['Date'])


,Date,flow,temp
0,2010-01-01,4330,5.3
1,2010-01-02,4700,5.2
2,2010-01-03,4820,5.2
3,2010-01-04,4840,5.2
4,2010-01-05,4730,5.2
5,2010-01-06,4670,5.0
6,2010-01-07,4560,4.8
7,2010-01-08,4600,5.0
8,2010-01-09,4580,5.0
9,2010-01-10,4560,5.1


In [36]:
def photo_period_new(year):
    hours_list = [] 
    doy = range(1,366)
    for days in doy:
    
        i = datetime.datetime(year, 1, 1) + datetime.timedelta(days - 1)
        date = i.strftime("%Y-%m-%d")
        url = "https://api.sunrise-sunset.org/json?lat=48.422006&lng=-122.340316&date=" + date
        response = requests.get(url)
        result = response.json()
        t = result['results']['day_length']
        (h, m, s) = t.split(':')
        hours = int(h) + int(m) * (1/60) #ignoring seconds
        hours_list.append(hours)
    df = pd.DataFrame({'doy' : doy, "photoperiod" : hours_list})
    return(df)
df_photo = photo_period_new(2010)

In [38]:
#that took a while to run so I am saving it into csv so that I can just read the csv later
df_photo.to_csv(data_string + "/photoperiod_skagit.csv")

In [52]:
df_temp_flow_subset['atu_solstice'] = df_temp_flow_subset['temp'].cumsum()
year = df_temp_flow_subset['Date'].dt.year[0]
atu_df_april = df_temp_flow_subset.loc[(df_temp_flow_subset['Date'] >= str(year)+'-4-01')]
atu_df_april['atu_april'] = atu_df_april['temp'].cumsum()

df_atu = df_temp_flow_subset.merge(atu_df_april[['Date','atu_april']], how = 'left', on='Date')
df_atu['flow_diff'] = df_atu['flow'].diff()
df_atu['temp_diff'] = df_atu['temp'].diff()
df_atu['doy'] = df_atu['Date'].dt.dayofyear

#adding photoperiod
df_atu_photo = df_atu.merge(df_photo, how = 'left', on='doy')


df_lunar = pd.DataFrame(pd.date_range(start='2009-12-21', end='2010-12-21'), columns = ['Date'])
df_lunar.head()
moon_list = []
for i in df_lunar['Date']:
    a=ephem.Moon(i)
    mn=a.moon_phase #illuminated percentage of the moon.
    moon_list.append(mn)

#adding lunar phase
df_lunar['lunar_phase'] = moon_list
df_lunar.head()
df_atu_photo_lunar = df_atu_photo.merge(df_lunar, how = 'left', on='Date')
df_atu_photo_lunar['photo_diff'] = df_atu_photo_lunar['photoperiod'].diff()
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(df_atu_photo_lunar)
    

C:\Users\maria\AppData\Local\Temp/ipykernel_24580/683901014.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp_flow_subset['atu_solstice'] = df_temp_flow_subset['temp'].cumsum()
C:\Users\maria\AppData\Local\Temp/ipykernel_24580/683901014.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  atu_df_april['atu_april'] = atu_df_april['temp'].cumsum()


,Date,flow,temp,atu_solstice,atu_april,flow_diff,temp_diff,doy,photoperiod,lunar_phase,photo_diff
0,2010-01-01,4330,5.3,5.3,NaN,NaN,NaN,1,8.466667,0.999403,NaN
1,2010-01-02,4700,5.2,10.5,NaN,370.0,-0.1,2,8.483333,0.979025,0.016667
2,2010-01-03,4820,5.2,15.7,NaN,120.0,0.0,3,8.500000,0.930183,0.016667
3,2010-01-04,4840,5.2,20.9,NaN,20.0,0.0,4,8.516667,0.856684,0.016667
4,2010-01-05,4730,5.2,26.1,NaN,-110.0,0.0,5,8.533333,0.764380,0.016667
5,2010-01-06,4670,5.0,31.1,NaN,-60.0,-0.2,6,8.566667,0.660109,0.033333
6,2010-01-07,4560,4.8,35.9,NaN,-110.0,-0.2,7,8.583333,0.550663,0.016667
7,2010-01-08,4600,5.0,40.9,NaN,40.0,0.2,8,8.616667,0.442048,0.033333
8,2010-01-09,4580,5.0,45.9,NaN,-20.0,0.0,9,8.633333,0.339157,0.016667
9,2010-01-10,4560,5.1,51.0,NaN,-20.0,0.1,10,8.666667,0.245782,0.033333


In [71]:
df_day_covariates = df_merged_day.merge(df_atu_photo_lunar, on = 'Date', how = 'left', validate = 'many_to_one')
df_night_covariates = df_merged_night.merge(df_atu_photo_lunar, on = 'Date', how = 'left', validate = 'many_to_one')

with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(df_day_covariates)


,date_range,Date,Down_time,Up_date,Up_time,midpoint,In,Up,Down,chinook0_hatchery_num,chinook0_wild_num,chinook1_hatchery_num,chinook1_wild_num,coho1_hatchery_num,coho1_wild_num,chum0_wild_num,pink0_wild_num,steelheadsmolt_wild_num,daytime_category,midpoint_doy,doy_x,year,flow,temp,atu_solstice,atu_april,flow_diff,temp_diff,doy_y,photoperiod,lunar_phase,photo_diff
0,2010-01-20 12:00:00,2010-01-20,NaN,NaT,NaN,NaT,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,day,NaN,20,2010,4980,4.9,100.8,NaN,70.0,0.0,20,9.016667,0.186283,0.033333
1,2010-01-21 12:00:00,2010-01-21,NaN,NaT,NaN,NaT,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,day,NaN,21,2010,5060,4.9,105.7,NaN,80.0,0.0,21,9.066667,0.266467,0.050000
2,2010-01-22 12:00:00,2010-01-22,08:40:00,2010-01-22,17:15:00,2010-01-22 12:57:30,8.583333,2010-01-22 17:15:00,2010-01-22 08:40:00,0.0,4.0,0.0,0.0,0.0,0.0,12.0,100.0,0.0,day,22.0,22,2010,5500,4.8,110.5,NaN,440.0,-0.1,22,9.100000,0.356833,0.033333
3,2010-01-23 12:00:00,2010-01-23,NaN,NaT,NaN,NaT,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,day,NaN,23,2010,5550,4.8,115.3,NaN,50.0,0.0,23,9.150000,0.455035,0.050000
4,2010-01-24 12:00:00,2010-01-24,NaN,NaT,NaN,NaT,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,day,NaN,24,2010,5350,4.8,120.1,NaN,-200.0,0.0,24,9.183333,0.558129,0.033333
5,2010-01-25 12:00:00,2010-01-25,07:40:00,2010-01-25,17:00:00,2010-01-25 12:20:00,9.333333,2010-01-25 17:00:00,2010-01-25 07:40:00,0.0,2.0,0.0,0.0,0.0,0.0,6.0,38.0,0.0,day,25.0,25,2010,4060,4.8,124.9,NaN,-1290.0,0.0,25,9.233333,0.662260,0.050000
6,2010-01-26 12:00:00,2010-01-26,NaN,NaT,NaN,NaT,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,day,NaN,26,2010,3530,4.8,129.7,NaN,-530.0,0.0,26,9.266667,0.762436,0.033333
7,2010-01-27 12:00:00,2010-01-27,NaN,NaT,NaN,NaT,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,day,NaN,27,2010,3540,4.6,134.3,NaN,10.0,-0.2,27,9.316667,0.852587,0.050000
8,2010-01-28 12:00:00,2010-01-28,08:00:00,2010-01-28,17:30:00,2010-01-28 12:45:00,9.500000,2010-01-28 17:30:00,2010-01-28 08:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,day,28.0,28,2010,4450,4.6,138.9,NaN,910.0,0.0,28,9.366667,0.926038,0.050000
9,2010-01-29 12:00:00,2010-01-29,NaN,NaT,NaN,NaT,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,day,NaN,29,2010,4440,4.7,143.6,NaN,-10.0,0.1,29,9.416667,0.976427,0.050000


In [77]:
df_day_covariates.to_csv(data_string+'skagit/2010_skagit_scoop_covariates_day.csv')
df_night_covariates.to_csv(data_string+'skagit/2010_skagit_scoop_covariates_night.csv')

In [78]:
#repeat same for scoop sheet